# 7장 실습

## Training and Test datasets

In [1]:
import tensorflow as tf

x_data = [[1, 2, 1], [1, 3, 2], [1, 3, 4], [1, 5, 5], [1, 7, 5], [1, 2, 5], [1, 6, 6], [1, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]
# Evaluation our model using this test dataset
x_test = [[2, 1, 1], [3, 1, 2], [3, 3, 4]]
y_test = [[0, 0, 1], [0, 0, 1], [0, 0, 1]]

X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 3])

W = tf.Variable(tf.random_normal([3,3]))
b = tf.Variable(tf.random_normal([3]))

hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
is_correct = tf.equal(prediction, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(201):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        if step % 40 == 0:
            print(step, cost_val, "\n", W_val)
    print("Prediction: ", sess.run(prediction, feed_dict={X: x_test}))
    print("Accuracy: ", sess.run(accuracy, feed_dict={X: x_test, Y: y_test}) )

0 3.7740567 
 [[2.158431   0.47066015 1.3451079 ]
 [0.30591714 2.2546668  1.6819485 ]
 [1.305886   0.7903211  1.3199083 ]]
40 0.73930514 
 [[1.929556   0.47506618 1.5695777 ]
 [1.3494393  1.5428791  1.3502139 ]
 [1.5138857  1.1806595  0.7215705 ]]
80 0.65777206 
 [[1.6892117  0.48323193 1.8017557 ]
 [1.5011973  1.3894634  1.3518717 ]
 [1.4577198  1.3420328  0.6163637 ]]
120 0.61802846 
 [[1.4955777 0.484256  1.9943663]
 [1.5424608 1.3661765 1.3338951]
 [1.4923072 1.3730806 0.5507289]]
160 0.58809984 
 [[1.3247104  0.48918188 2.1603076 ]
 [1.5572773  1.3697305  1.3155249 ]
 [1.5443687  1.3749359  0.49681196]]
200 0.5636022 
 [[1.1682656  0.49871573 2.3072195 ]
 [1.5639051  1.3777821  1.3008448 ]
 [1.5987188  1.3697886  0.44760945]]
Prediction:  [2 2 2]
Accuracy:  1.0


## Normalized inputs (min-max scale)

In [2]:
import tensorflow as tf
import numpy as np

xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
[823.02002, 828.070007, 1828100, 821.655029, 828.070007],
[819.929993, 824.400024, 1438100, 818.97998, 824.159973],
[816, 820.958984, 1008100, 815.48999, 819.23999],
[819.359985, 823, 1188100, 818.469971, 818.97998],
[819, 823, 1198100, 816, 820.450012],
[811.700012, 815.25, 1098100, 809.780029, 813.669983],
[809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

def MinMaxScaler(data):
    numerator = data - np.min(data, 0) 
    denominator = np.max(data, 0) - np.min(data, 0) 
    # noise term prevents the zero division 
    return numerator / (denominator + 1e-7) 

xy = MinMaxScaler(xy)

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.float32, [None, 1])

W = tf.Variable(tf.random_normal([4,1]))
b = tf.Variable(tf.random_normal([1]))

hypothesis = tf.matmul(X, W) + b;
cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 100 == 0:
        print(step, cost_val, "\n", hy_val)

0 0.15411657 
 [[ 1.3744984 ]
 [ 0.31607148]
 [ 0.30599186]
 [ 0.23020753]
 [ 0.4229991 ]
 [ 0.33310834]
 [-0.1961527 ]
 [-0.69301105]]
100 0.15375268 
 [[ 1.3752881 ]
 [ 0.3170832 ]
 [ 0.30682537]
 [ 0.2308422 ]
 [ 0.423736  ]
 [ 0.3338345 ]
 [-0.19556576]
 [-0.69235325]]
200 0.15339038 
 [[ 1.376077  ]
 [ 0.31809407]
 [ 0.30765802]
 [ 0.23147607]
 [ 0.4244722 ]
 [ 0.33455992]
 [-0.19497964]
 [-0.6916963 ]]
300 0.15303046 
 [[ 1.3768619 ]
 [ 0.31910133]
 [ 0.3084873 ]
 [ 0.23210692]
 [ 0.4252051 ]
 [ 0.3352822 ]
 [-0.19439599]
 [-0.69104147]]
400 0.15267277 
 [[ 1.3776388 ]
 [ 0.32010287]
 [ 0.30931193]
 [ 0.23273408]
 [ 0.4259336 ]
 [ 0.3360005 ]
 [-0.19381413]
 [-0.6903877 ]]
500 0.15231733 
 [[ 1.3784139 ]
 [ 0.32110208]
 [ 0.31013435]
 [ 0.23335916]
 [ 0.42665988]
 [ 0.3367166 ]
 [-0.19323444]
 [-0.6897361 ]]
600 0.15196326 
 [[ 1.3791878 ]
 [ 0.32210082]
 [ 0.310956  ]
 [ 0.23398346]
 [ 0.4273854 ]
 [ 0.33743197]
 [-0.1926555 ]
 [-0.68908525]]
700 0.15161145 
 [[ 1.3799598 ]
 [ 0